# Graphen

Ein **Graph** besteht aus **Knoten** und **Kanten**. Er kann gerichtet oder ungerichtet sein. Die Kanten 
können **gewichtet** sein (Kosten).  

![a](img/graphen_01.png)
![a](img/graphen_02.png)

Mit Graphen können binäre Beziehungen zwischen Objekten modelliert werden. Die Objekte werden
durch die Knoten, die Beziehungen durch die Kanten modelliert.  


* Orte: Entfernungen, Dauer
* Personen: ist befreundet, verheiratet mit
* Ereignisse: a muss vor b geschehen

#### Begriffe

![a](img/graphen_03.png)
![a](img/graphen_04.png)

* x ist zu y adjazent, x und y sind Nachbarn
* der Grad von y ist 2
* a ist Vorgänger von b, b ist Nachfolger von a
* der Eingangsgrad von b ist 2, der Ausgangsgrad ist 1
* Ein **Weg** ist eine Folge von adjazenten Knoten
* Ein **Kreis** ist ein Weg, der zurück zum Startknoten führt.
 

### Implementation von Graphen

Eine Implementation sollte zwei Dinge zügig können:
* Die Frage klären, ob es eine Kante von Knoten a nach Knoten b gibt,
* Alle Nachbarn von a durchlaufen.


#### Adjazenzmatrix eines ungewichteten Graphen
Wir ordnen jedem Knoten einen Index zu. Die erste Reihe der Adjazenzmatrix gibt die Nachbarn des Knoten mit Index 0 an.

![a](img/graphen_05.png)

In [ ]:
a b c d   # Knoten
0 1 2 3   # Index

In [2]:
G = [[0, 1, 1, 0],  
     [0, 0, 0, 1],  
     [0, 1, 0, 1],  
     [0, 1, 0, 0]]

In [3]:
# Gibt es Kante von a nach b?
G[0][1] == 1  

True

In [8]:
# alle Nachbarn von a  
[j for j in range(len(G)) if G[0][j] == 1]          

[1, 2]

In [10]:
# alle Nachbarn von a  (Buchstaben)
[chr(j+97) for j in range(len(G)) if G[0][j] == 1]  

['b', 'c']

#### Adjazenzmatrix eines gewichteten Graphen

![a](img/graphen_07.png)

In [ ]:
a b c d   # Knoten
0 1 2 3   # Index

Nicht vorhandene Knoten haben unendliche Kosten.

In [9]:
inf = float('inf') 
G = [[0,   8,   2, inf],  
     [inf, 0, inf,   4],  
     [inf, 1,   0,   6],  
     [inf, 2, inf,   0]]

In [10]:
# Kosten von a nach b
G[0][1]

8

In [12]:
# alle Nachbarn von a
[j for j in range(len(G)) if j!=0 and G[0][j] < inf]

[1, 2]

#### Analyse der Implementation durch eine Adjazenzmatrix

$V$ = Knoten (vertices), $E$ = Kanten (Edges)

* Platzbedarf = $O(|V|^2)$
* direkte Zugriff auf Kante $(i,j)$ in konstanter Zeit möglich
* Liste der Nachbarn in $O(|V|)$ - bei dünn besetzten Graphen eher schlecht

#### Der Algorithmus von Floyd-Warshall

Der Algorithmus von Floyd-Warshall löst das **all-pairs-shortest-path**-Problem

![a](img/graphen_08.png)

Die Matrix $D^k$ enthält die Kosten der kürzesten Wege zwischen zwei Knoten $i$ und $j$, die als Zwischenknoten
nur die Knoten $0,1,..., k$ verwenden. Dann lässt sich $d_{i,j}^{k}$ errechnen durch das Minimum von 
 $d_{i,j}^{k-1}$ und  $d_{i,k}^{k-1} +  d_{k,j}^{k-1}$.
 
Um die Kantenfolge zu rekonstruieren, wird gleichzeitig eine Folge von Matrizen $P^k$ aufgebaut, die an Position 
$p_{i,j}^{k}$ den vorletzten Knoten auf dem kürzesten Weg von i nach j notiert, der nur über die Zwischenknoten 
$0,1,..., k$ läuft.
 


![a](img/graphen_09.png)

In [3]:
def floyd(c):
    n = len(c)
    d = [[0]*n for j in range(n)]   # Matrix D
    p = [[0]*n for j in range(n)]   # Matrix P
    for i in range(n):
        for j in range(n):
            d[i][j] = c[i][j]
            p[i][j] = i

    for k in range(n):
        for i in range(n):
            for j in range(n):
                tmp = d[i][k] + d[k][j]
                if tmp < d[i][j]:
                    d[i][j] = tmp
                    p[i][j] = p[k][j]
    return d, p

def getPath(p, i, j):
    if i == j: return chr(i+97)
    return getPath(p,i,p[i][j]) + ' - ' + chr(j+97)

def printMatrix(a):
    for i in range(len(a)):
        for j in range(len(a)):
            if a[i][j] == inf:
                print("{:4}".format("."),end='')
            else:
                print("{:<4}".format(a[i][j]),end='')
        print()
    print()

inf = float('inf')
G = [[0,   8,   2, inf],
     [inf, 0, inf,   4],
     [inf, 1,   0,   6],
     [inf, 2, inf,   0]]

d, p = floyd(G)

print("Gegebenen Kostenmatrix:")
printMatrix(G)

print("Errechnete Distanzmatrix:")
printMatrix(d)

print("Errechnete Wegematrix:")
printMatrix(p)

v, w = 'a','b'        # Start und Zielknoten
vi, wi = ord(v)-ord('a'), ord(w)-ord('a')   # Index von Start und Zielknoten
print("Kürzester Weg von",v,"nach",w)
print(getPath(p,vi,wi))
print("Kosten =",d[vi][wi])

Gegebenen Kostenmatrix:
0   8   2   .   
.   0   .   4   
.   1   0   6   
.   2   .   0   

Errechnete Distanzmatrix:
0   3   2   7   
.   0   .   4   
.   1   0   5   
.   2   .   0   

Errechnete Wegematrix:
0   2   0   1   
1   1   1   1   
2   2   2   1   
3   3   3   3   

Kürzester Weg von a nach b
a - c - b
Kosten = 3


Laufzeit des Floyd-Warshall-Algorithmus: $O(|V|^3)$   

#### Implementation eines Graphen durch Adjazenzlisten

Jedem Knoten wird eine Liste seiner Nachbarn zugeordnet. Wir werden statt Listen *sets* oder *dictionaries* verwenden.

Ein ungewichteter Graph wird durch ein dictionary implementiert, mit den Knoten als key und der Nachbarmenge als value.

![a](img/graphen_10.png)

In [4]:
G = {'a':set('bc'),
     'b':set('d'), 
     'c':set('bd'),
     'd':set('b')}

In [5]:
# gibt es Kante von a nach b   
'b' in G['a']

True

In [6]:
# alle Nachbarn von a: 
G['a']

{'b', 'c'}

In [7]:
# alle Knoten von G durchlaufen:
for v in G:
    print(v)

a
b
c
d


Ein gewichteter Graph wird durch ein dictionary implementiert, mit den Knoten als key und einem dictionary, in dem die Kosten zu den Nachbarn verzeichnet sind, als value.

![a](img/graphen_11.png)

In [8]:
G = {'a': {'b':8, 'c':2},          
     'b': {'d':4},               
     'c': {'b':1, 'd':6},          
     'd': {'b':2}}

In [9]:
# gibt es Kante von a nach b  
'b' in G['a']  

True

In [10]:
# die Kosten der Kante von a nach b:  
G['a']['b']     

8

In [11]:
# alle Nachbarn von a  
[v for v in G['a']]

['b', 'c']

In [12]:
# alle Knoten von G durchlaufen:   
for v in G:   
    print(v)

a
b
c
d


#### Analyse Implementation durch Adjazenzmatrix

* Platzbedarf = $O(|E|)$
* direkte Zugriff auf Kante $(i,j)$ in konstanter Zeit nicht möglich
* sinnvoll bei eher dünn besetzten Graphen.
* sinnvoll bei Algorithmen, die, gegeben einen Knoten v, dessen Nachbarn verarbeiten müssen.

### Erreichbarkeit

![a](img/graphen_10.png)

Gegeben ein Graph G und ein Startknoten s. Welche Knoten sind von s erreichbar?

In [13]:
G = {'a':set('bc'),
     'b':set('d'), 
     'c':set('bd'),
     'd':set('b')}

```
Setze alle Knoten auf nicht besucht
explore(s)
Gib alle Knoten aus, die besucht wurden

def explore(v): 
    merke v als besucht
    für alle Nachbarn w von v:
        wenn w nicht besucht:
            explore(w)
```

In [14]:
visited =  {v : False for v in G}       
def explore(v):  
    visited[v] = True
    for w in G[v]:
        if not visited[w]:
            explore(w) 
            
s = 'c' 
explore(s)
result = [v for v in G if visited[v]]            
print(*result)

b c d


Die Funktion **explore** realisiert eine rekursive Tiefensuche (depth first search, dfs). <br>
Laufzeit: Jeder Knoten wird höchstens einmal explored $O(|V|)$, jeder Knoten prüft seine Nachbarn. Die Zahl der Nachbarn liegt in $O(|E|)$, also insgesamt $O(|V|) + O(|E|)$

### Zusammenhangskomponenten eines ungerichteten Graphen

Die Knoten eines ungerichteten Graphen G können in **Zusammenhangskomponenten** (Connected Components) aufgeteilt werden, so dass v von w genau dann erreichbar ist, wenn beide in derselben Zusammenhangskomponente liegen.

![a](img/graphen_12.png)

In [15]:
G = {
    'a': set('e'),
    'b': set('e'),
    'c': set('d'),
    'd': set('c'),
    'e': set('abf'),
    'f': set('e')
}

def explore(v):
    visited[v] = True
    ccnum[v] = cc
    for w in G[v]:
        if not visited[w]:
            explore(w)

visited = {v : False for v in G}
ccnum = {v : 0 for v in G}     # connected component nr von v
cc = 1                         # aktuelle connected component nr
for v in G:
    if not visited[v] :
        explore(v)
        cc+=1

for i in range(1,cc):
    result = [v for v in G if visited[v] and ccnum[v] == i]
    print(i,'-',*result)

1 - a b e f
2 - c d


### Previsit und postvisit Nummern der Tiefensuche

DFS endet damit, dass alle Knoten markiert sind. Das ist
als Ergebnis noch nicht sonderlich interessant. Interessant wird 
DFS daduch, dass man bei der Ausführung noch Daten sammelt, z.B. die **previsit** und **postvisit** Nummern.

![a](img/graphen_13.png)

Für alle Knoten u und v gilt: die Intervalle [pre(u),post(u)] und [pre(v),post(v)] sind entweder
ineinander verschachtelt oder disjunkt.

In [36]:
G = {
'a': set('bcd'),
'b': set('ac'),
'c': set('ab'),
'd': set('a'),
'e': set('f'),
'f': set('e'),
'g': set('hi'),
'h': set('gi'),
'i': set('gh'),
}


def explore(v):
    global counter
    visited[v] = True
    previsit[v] = counter
    counter += 1
    for w in sorted(G[v]):  #  alphabetische Reihenfolge
        if not visited[w]:
            explore(w)
    postvisit[v] = counter
    counter += 1

visited = {v : False for v in G}
previsit = {v : 0 for v in G}
postvisit = {v : 0 for v in G}
counter = 1

for v in G.keys():
    if not visited[v] :
        explore(v)

for v in sorted(G.keys(),key=lambda v: previsit[v]):
    print("{:2} {:2} {:2}".format(v,previsit[v],postvisit[v]))

a   1  8
b   2  5
c   3  4
d   6  7
e   9 12
f  10 11
g  13 18
h  14 17
i  15 16


### Topologische Sortierung

Wir wollen eine Nummerierung der Knoten finden, die mit den Kantenrichtungen
kompatibel ist.

![a](img/graphen_14.png)         

Es gilt: ein Graph kann genau dann topologisch sortiert werden, wenn er keinen Kreis enthält.
Genau die **gerichteten azyklischen Graphen (directed acyclic graph, DAG**) können topologisch sortiert werden. 

Eine **Quelle (source)** ist ein Knoten mit dem Eingangsgrad 0. Eine **Senke (sink)** ist ein Knoten mit dem Ausgangsgrad 0.


Idee für die topologische Sortierung (1. Versuch):

In [ ]:
Solange der Graph nicht leer:
    Folge einem Pfad bis es nicht mehr weitergeht.
    Finde dort eine Senke
    Füge die Senke in die Sortierung wie in einen Stapel ein
    Entferne die Senke aus dem Graphen.

Laufzeit:  $O(\left|V\right|^2)$    -   nicht gut

Verbesserungsmöglichkeit: statt den Pfad immer wieder von vorne zu beginnen, wird nach der
Entfernung der Senke der Weg so weit wie nötig zurückgegangen. Das ist Tiefensuche! Die abwärts sortierte Reihe der postvisit-Nummern ergibt eine topologische Sortierung.

![a](img/graphen_16.png)

In [4]:
G = {
'a': set('c'),
'b': set('c'),
'c': set('d'),
'd': set('g'),
'e': set('df'),
'f': set('g'),
'g': set()
}

def explore(v):
    global counter
    visited[v] = True
    for w in G[v]:
        if not visited[w]:
            explore(w)
    postvisit[v] = counter
    counter += 1
    
visited = {v : False for v in G}
postvisit = {v : 0 for v in G}
counter = 1

for v in G:
    if not visited[v] :
        explore(v)

topolist = sorted(G.keys(),key=lambda v: postvisit[v],reverse = True)
for i in range(len(topolist)):
    print(i+1,topolist[i])

1 e
2 f
3 b
4 a
5 c
6 d
7 g


### Starke Zusammenhangskomponenten

In einem **gerichteten** Graphen heißen zwei Knoten u und v **zusammenhängend**, wenn u von v und v von u erreichbar ist.           
Ein gerichteter Graph kann partitioniert werden in **starke Zusammenhangskomponenten (strongly connected components, SCC)**, wobei zwei Knoten genau dann zusammenhängen, wenn sie in der selben Komponente sind.

![b](img/graphen_17.png)
![a](img/graphen_18.png)

Der **Metagraph** zeigt, wie die starken Zusammenhangskomponenten untereinander verbunden sind. Der Metagraph ist immer ein DAG.

![c](img/graphen_19.png)

Wenn v in einer Senke des Metagraphen liegt, dann findet explore(v) die Zusammenhangskomponente.

Es gilt: Wenn C und C' zwei starke Zusammenhangskomponenten sind und es eine Kante von einem Knoten aus C nach einem Knoten aus C' gibt, dann ist  die größte postvisit-Nummer in C größer als die größte postvisit-Nummer in C'.  

Folgerung: der Knoten mit der größten postvisit-Nummer ist in einer SCC-Quelle. Wir suchen aber eine SCC-Senke.    

Dazu betrachten wir den reversen Graphen. Den reversen Graphen $G^R$ eines gerichteten Graphen G erhält man, wenn
man die Kantenrichtungen umdreht.  

$G$ und $G^R$ haben dieselben SCCs.  
SCC-Quellen in $G^R$ sind SCC-Senken in $G$.


In [ ]:
# Bestimmung der starken Zusammenhangskomponenten in einem gerichteten Graphen G
 
Führe Tiefensuche auf reversen Graphen durch und merke postvisit-Nummern.
Für alle Knoten v in G in umgekehrter postvisit-Nummerierung:
    Falls v noch nicht besucht:
         explore(v) und markiere alle besuchten Knoten als neuen SCC.

* Bestimmung der postvisit-Nummern des reversen Graphen:

![a](img/graphen_20.png)
![a](img/graphen_21.png)

Umgekehrte Reihenfolge der postvisit-Nummern: f i h g a e b c d

* Bestimmung der SCCs mit Tiefensuche in der Reihenfolge  f i h g a e b c d.

![a](img/graphen_22.png)
![a](img/graphen_23.png)

In [5]:
G = {
'a': set('b'),
'b': set('ef'),
'c': set('b'),
'd': set('ag'),
'e': set('ach'),
'f': set(),
'g': set('h'),
'h': set('i'),
'i': set('fh')
}

# explore reverse Graph, merke postvisit nr
def exploreR(v):
    global counter
    visited[v] = True
    for w in Gr[v]:
        if not visited[w]:
            exploreR(w)
    postvisit[v] = counter
    counter+=1

# explore Graph, merke connected component number
def explore(v):
    visited[v] = True
    ccnum[v] = cc
    for w in G[v]:
        if not visited[w]:
            explore(w)

# Gr ist der reverse Graph von G
Gr = {v:set() for v in G}
for v in G:
    for w in G[v]:
        Gr[w].add(v)

# dfs durch den reversen Graphen, merke postvisit-nr
visited = {v : False for v in Gr}
postvisit = {v : 0 for v in Gr}
counter = 1
for v in Gr:
    if not visited[v] :
        exploreR(v)

# die Knoten in der umgekehrten postvisit-Reihenfolge des reversen Graphen
vlist = sorted(Gr,key=lambda v: postvisit[v],reverse=True)

# dfs durch den Ausgangsgraphen, in der vlist Reihenfolge, dabei component nr merken
visited = {v : False for v in G}
ccnum = {v : 0 for v in G}
cc = 1
for v in vlist:
    if not visited[v] :
        explore(v)
        cc+=1

# Ausgabe
for i in range(1,cc):
    result = [v for v in G if ccnum[v] == i]
    print(i,'-',*result)

1 - f
2 - h i
3 - g
4 - a b c e
5 - d


### Kürzeste Wege

Die Länge eines Pfades in einem ungewichteten Graphen ist die Anzahl seiner Kanten. Die Entfernung zweier Knoten 
in einem ungewichteten Graphen ist die Länge des kürzesten Pfades zwischen ihnen.

![b](img/graphen_24.png)

Die Entfernung zwischen S und H ist 3.

Die **Breitensuche (breadth first search, bfs)** traversiert den Graphen in Schichten nach der Entfernung zum Ausgangsknoten.
Zu jedem Knoten merkt man sich seinen Vorgänger (prev), dadurch entsteht ein **shortest-path Baum**.
(Ein Baum ist ein zusammenhängender Graph, der keine Kreise enthält). Daraus lässt sich der kürzeste
Weg vom Ausgangsknoten zu jedem anderen Knoten rekonstruieren.

![a](img/graphen_25.png)

In [ ]:
# Breitensuche mit Ausgangsknoten s:
Für jeden Knoten u in G: 
    dist(u) = unendlich 
    prev(u) = None

Füge s in eine Schlange Q ein 
dist(s) = 0

Solange Q nicht leer:
    hole Knoten u aus der Schlange
        für alle Nachbarn v von u:
            falls dist(v) unendlich:
                Füge v in die Schlange Q ein
                dist(v) = dist(u) + 1
                prev(v) = u

In [1]:
G = {
'a': set('sb'),
'b': set('acgh'),
'c': set('bs'),
'd': set('sef'),
'e': set('sd'),
'f': set('dg'),
'g': set('bfh'),
'h': set('bg'),
's': set('acde')
}

def reconstructPath(s,u,prev):
    result = []
    while u != s:
        result.append(u)
        u = prev[u]
    result.append(s)
    result.reverse()
    return result

inf = float('inf')
dist = {v:inf for v in G}
prev = {v:None for v in G}

s = 's'         # Startknoten
dist[s] = 0
Q = [s]         # Queue mit pop(0) und append
while Q:
    u = Q.pop(0)
    for v in G[u]:
        if dist[v] == inf:
            Q.append(v)
            dist[v] = dist[u]+1
            prev[v] = u
ziel = 'h'
print('Pfad von',s,'nach',ziel,':',*reconstructPath(s,ziel,prev))
print('Länge =',dist[ziel])

Pfad von s nach h : s c b h
Länge = 3


### Der Algorithmus von Dijkstra

Der Algorithmus von Dijkstra findet  in einem gerichteten, mit nichtnegativen Kosten gewichteten
Graphen die kürzesten Wege von einem Startknoten zu allen anderen Knoten. Er löst das
**single source shortest paths**-Problem.

![a](img/graphen_26.png)

In [ ]:
# Algorithmus von Dijkstra
Setze dist des Startknotens s vorläufig auf 0,
    alle anderen vorläufig auf unendlich. 
Setze prev aller Knoten auf None
Solange es noch vorläufige Knoten gibt
    setze dist des billigsten vorläufigen Knoten auf endgültig
    Für jeden Nachbarn v von u:   
        dist(v) = min(dist(v),dist(u) + Kosten von u nach v)   # Relaxiere Kante (u,v) 
        prev(v) = letzter Knoten auf dem Weg zu den minimalen Kosten

In [2]:
G = {
'a': {'b':2, 'c':9},
'b': {'c':5, 'd':13},
'c': {'d':6, 'e':9},
'd': {'e':1, 'f':5},
'e': {'f':2},
'f': {}
}

from heapq import heapify, heappop, heappush

def reconstructPath(s,u,prev):
    temp = []
    while u != s:
        temp.append(u)
        u = prev[u]
    temp.append(s)
    temp.reverse()
    return temp

inf = float('inf')
dist = {v:inf for v in G}
prev = {v:None for v in G}

s = 'a'         # startknoten
dist[s] = 0
visited = set()

heap = [(dist[v],v) for v in G]
heapify(heap)

while heap:
    _, u = heappop(heap)
    if u in visited: continue
    visited.add(u)
    for v in G[u]:
        if dist[v] > dist[u] + G[u][v]:
            dist[v] = dist[u] + G[u][v]
            prev[v] = u
            heappush(heap,(dist[v],v))
ziel = 'f'
print('Pfad von',s,'nach',ziel,':',*reconstructPath('a','f',prev))
print('Distanz:',dist[ziel])

Pfad von a nach f : a b c d e f
Distanz: 16


Während des Algorithmus müssen wir uns aus den vorläufig markierten Knoten laufend einen billigsten suchen. 
Dazu nutzen wir einen Heap. 
Die Laufzeit hängt ab vom Aufwand der Operationen zur Entnahme der Knoten aus dem Heap.
Bei einen Fibonacci-Heap ist dies $O(1)$ und der Aufwand also $O(\left|V\right|+\left|E\right|)$.
Aufwand bei einem Min-Heap: $O((\left|V\right|+\left|E\right|)\cdot\log\left|V\right|)$.

### Der Algorithmus von Bellman-Ford

Der Algorithmus von **Bellman-Ford** kann auch mit negativen Kantengewichten umgehen, vorausgesetzt
es gibt keine Kreise mit negativem Gewicht.



In [ ]:
# Algorithmus von Bellman-Ford
Setze dist des Startknotens auf 0,
    alle anderen auf unendlich. 
Setze prev aller Knoten auf None
Solange sich was ändert:      # höchstens |V| - 1 mal
    Fur alle Kanten (u,v):
        Relaxiere(u,v)        # d.h. ggf. Kosten von v via u-v verbessern

Laufzeit: $O(\left|V\right|\cdot\left|E\right|)$.

![a](img/graphen_27.png)



In [ ]:
# Durchgänge bis sich nichts mehr ändert
0 :  a:0 b:inf c:inf d:inf e:inf
1 :  a:0 b:6 c:4 d:2 e:7
2 :  a:0 b:2 c:4 d:2 e:7
3 :  a:0 b:2 c:4 d:-2 e:7
4 :  a:0 b:2 c:4 d:-2 e:7

In [6]:
G = {
'a': {'b':6,'e':7},
'b': {'d':-4,'e':8,'c':5},
'c': {'b':-2},
'd': {'c':7},
'e': {'c':-3,'d':9}
}

def reconstructPath(s,u,prev):
    temp = []
    while u != s:
        temp.append(u)
        u = prev[u]
    temp.append(s)
    temp.reverse()
    return temp

inf = float('inf')
dist = {v:inf for v in G}
prev = {v:None for v in G}

start, ziel = 'a' , 'b'
dist[start] = 0

changed = True
while changed:
    changed = False
    for u in G:
        for v in G[u]:
            if dist[v] > dist[u] + G[u][v]:
                dist[v] = dist[u] + G[u][v]
                prev[v] = u
                changed = True

print('Pfad von',start,'nach',ziel,':',*reconstructPath(start,ziel,prev))
print('Distanz:',dist[ziel])

Pfad von a nach b : a e c b
Distanz: 2


### Minimale Spannbäume

* Ein Teilgraph H eines ungerichteten Graphen G heisst **Spannbaum** von G, wenn H ein Baum auf den Knoten von
G ist. 

2. Ein Spannbaum S eines gewichteten, ungerichteten Graphen heisst **minimaler Spannbaum, (minimal spanning tree, MST)**, wenn S minimales Gewicht unter allen Spannbäumen von G besitzt.



#### Beispiele:
Man möchte Computer kostengünstig vernetzen.

![b](img/graphen_28.png)
![b](img/graphen_29.png)

Man möchte Orte mit möglichst kurzen Straßen verbinden.

![a](img/graphen_30.png) &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    ![a](img/graphen_31.png)

 

#### Eigenschaften von Bäumen

* Ein Baum ist ein zusammenhängender, ungerichteter Graph, der keine Kreise enhält.
* Ein Baum mit $n$ Knoten hat $n-1$ Kanten.
* Jeder zusammenhängende, ungerichtete Graph mit $\left|V\right| = \left|E\right| - 1$ ist ein Baum.
* Ein ungerichteter Graph ist genau dann ein Baum, wenn es zwischen je zwei Knoten einen eindeutigen Pfad gibt.

### Der Algorithmus von Kruskal

Füge immer wieder die leichteste Kante hinzu, vorausgesetzt es entsteht kein Kreis.

![b](img/graphen_32.png)
![b](img/graphen_33.png)

a-d (5), c-e (5), d-f (6), a-b (7), b-e (7), e-g (9), Gesamtkosten:  39

Um zu verhindern, dass ein Kreis entsteht, wird jedem Knoten ein Repräsentant ('Chef') zugeordnet.
Eine Kante wird nur in den MST aufgenommen, wenn die beteilgten Knoten nicht denselben Chef haben.

In [ ]:
a-d: 5   Chef von a wird d
c-e: 5   Chef von c wird e
d-f: 6   Chef von d wird f
a-b: 7   Chef von f wird b
b-e: 7   Chef von b wird e
e-g: 9   Chef von e wird g

![a](img/graphen_34.png)

Um lange Pfade im Chefbaum zu verhindern, wird der Algorithmus mit **union by rank** und **path compression** optimiert.

Wird der Chef von a gesucht, dann werden alle Zwischenknoten auf dem Weg zum Chef direkt mit
diesem verbunden. Jeder Knoten erhält einen Rang. Bei der Neuzuweisung eines
Chefs wird der Knoten mit dem höheren Rang Chef. Bei Gleichheit wird einer gewählt, dessen Rang
dann erhöht wird.

In [ ]:
a-d: 5   Rang a gleich Rang d : Chef von a wird d
Rang von d wird 1
c-e: 5   Rang c gleich Rang e : Chef von c wird e
Rang von e wird 1
d-f: 6   Rang d größer Rang f : Chef von f wird d
a-b: 7   Rang d größer Rang b : Chef von b wird d
b-e: 7   Rang d gleich Rang e : Chef von d wird e
Rang von e wird 2
Kompression: Chef von b wird e
Kompression: Chef von f wird e
e-g: 9   Rang e größer Rang g : Chef von g wird e

![b](img/graphen_35.png)

In [8]:
G = {
  'a': {'b':7, 'd':5},
  'b': {'a':7, 'd':9, 'e':7, 'c':8},
  'c': {'b':8, 'e':5},
  'd': {'a':5, 'b':9, 'e':15, 'f':6},
  'e': {'b':7, 'c':5, 'd':15, 'f':8, 'g':9},
  'f': {'d':6, 'e':8, 'g':11},
  'g': {'e':9, 'f':11}
}

def find(u):
    if chef[u] != chef[chef[u]]:
        chef[u] = find(chef[u])        # Path compression
    return chef[u]

def union(u, v):                       # union by rank
    u, v = find(u), find(v)
    if rank[u] > rank[v]:
        chef[v] = u
    else:
        chef[u] = v
    if rank[u] == rank[v]:
        rank[v]+=1

summe = 0
gesehen = set()

E = [(G[u][v],u,v) for u in G for v in G[u]]  # Kantenliste mit Kosten

mst = []                             # leerer Anfangsbaum
chef = {u:u for u in G}              # zunächst ist jeder eigener chef
rank = {u:0 for u in G}              # und hat Rang 0
for _, u, v in sorted(E):
    if find(u) != find(v):           # wenn chefs verschieden, wird die
        mst.append((u, v))           # Kante in den mst aufgenommen
        summe += G[u][v]
        union(u, v)

print(mst)
print('Gesamtkosten: ', summe)

[('a', 'd'), ('c', 'e'), ('d', 'f'), ('a', 'b'), ('b', 'e'), ('e', 'g')]
Gesamtkosten:  39


### Der Algorithmus von Jarnik-Prim

Gehe von einem Knoten aus und füge immer wieder einen neuen Knoten entlang der leichtesten Kante hinzu.

In [1]:
from heapq import heappop, heappush

G = {
  'a': {'b':7, 'd':5},
  'b': {'a':7, 'd':9, 'e':7, 'c':8},
  'c': {'b':8, 'e':5},
  'd': {'a':5, 'b':9, 'e':15, 'f':6},
  'e': {'b':7, 'c':5, 'd':15, 'f':8, 'g':9},
  'f': {'d':6, 'e':8, 'g':11},
  'g': {'e':9, 'f':11}
}

start = 'a'                              # Startknoten
mst = {}                                 # leere Teillösung
heap = [(0, None, start)]                # Heap mit Kanten und Kosten
summe = 0
while heap:
    cost, prev, u = heappop(heap)
    if u in mst: continue            # Zielknoten schon im Baum?
    mst[u] = prev                    # Kante von u nach prev kommt in mst
    summe += cost
    for v, cost in G[u].items():
        heappush(heap, (cost, u, v))

print(mst)
print('Gesamtkosten: ', summe)

{'a': None, 'd': 'a', 'f': 'd', 'b': 'a', 'e': 'b', 'c': 'e', 'g': 'e'}
Gesamtkosten:  39
